In [ ]:
import numpy as np
from collections import defaultdict
import json

In [ ]:
draft_pick_movs = [[0.36211491756677816, 1.3170180246826415, 0.651214291927777],
 [0.2823684338888295, 1.85110557120951, 0.5950874564833337],
 [0.23291861205436207, 2.4053508391966627, 0.5788155686897763],
 [0.23259802980965155, 2.8807892108432966, 0.5309845200620787],
 [0.21006429689361383, 3.2163877631533726, 0.5236513566775773]]

age_shift ={19: 6.4,
 20: 6.7,
 21: 5.0,
 22: 4.4,
 23: 2.2,
 24: 2.0,
 25: 0.1,
 26: -0.1,
 27: -1.2,
 28: -1.4,
 29: -2.0,
 30: -2.7,
 31: -3.6,
 32: -3.7,
 33: -3.7,
 34: -4.7,
 35: -5.1}

max_shift = max(age_shift.values())


draft_salaries = defaultdict(lambda: 750,
{0: 7500,
 1: 6750,
 2: 6000,
 3: 5250,
 4: 4500,
 5: 4150,
 6: 3750,
 7: 3400,
 8: 3000,
 9: 2850,
 10: 2700,
 11: 2550,
 12: 2400,
 13: 2250,
 14: 2100,
 15: 1950,
 16: 1800,
 17: 1650,
 18: 1500,
 19: 1500,
 20: 1500,
 21: 1500,
 22: 1500,
 23: 1500,
 24: 1500,
 25: 1500,
 26: 1500,
 27: 1500,
 28: 1500,
 29: 1500,
})


iv = np.array([73.2 , 60.1 , 54.2 , 38.7 ,  3.95,  2.06, -0.92,  1.1 , 12.5 ,
       11.3 , 14.3 , 10.4 ])
def get_mov(x):
    offset = len(iv)//3
    tot = -2.96
    for i in range(offset):
        tot += (iv[offset+i])*(np.tanh((x-iv[i])/iv[2*offset+i])+1.0)
    return tot

# estimated mov per ovr
o2m = {}
for i in np.arange(101):
    o2m[i] = get_mov(i)

# replacement level
r_lvl = -1.44

# salaries to mov
sA,sB = (3.412670359422221,0)
player_vs_cap = {
0 : [-14.86,  1.02,   0.00] ,
1 : [-9.88,  0.76,  1.06] ,
2 : [-6.65,  0.51,  0.82] ,
3 : [-3.09,  0.06,  0.53] ,
4 : [-0.06,  0.01,  0.3 ] ,

}

In [ ]:
team = { 'p' : [(22,66,22000,2),(22,66,22000,2),(22,66,22000,2),(22,66,22000,2)], #(age,ovr,salary,years_left)
         'd' : [(1,0,3),(2,0,3),(1,1,3),(2,1,3),(1,2,3),(2,2,3),(1,3,3),(2,3,3),(1,4,3),(2,4,3)] #(round,years_left,team_MOV)
       }
def get_team_value(team,sCap=90000,minS=750):
    sCapS = sCap/90000

    # turn mov into draft pick and future mov
    m2pos = lambda x: int(round(np.clip(14.5000 +  -1.7672*x,0,29)))
    m2next = lambda year,mov: [1,0.5,0.25,0.08,0.03][year]*mov 

    # turn draft picks into specific predictions
    draft_picks = [ (yr,m2pos(m2next(yr,mov))+30*(rnd-1)) for rnd,yr,mov in team['d'] if yr < 5]

    # for this and the next 4 years
    pars = defaultdict(list)
    tss = defaultdict(int)

    for i in range(5):
        for age,ovr,con,yrl in team['p']:
            if yrl >= i: # are we on the team still?
                tss[i] += con
                for j in range(i):
                    ovr+=age_shift.get(age+j,max_shift)
                pars[i].append(o2m[round(np.clip(ovr,0,100))])

        for yr,p in draft_picks:
            if yr+1 <= i: # if drafted
                dsal = sCapS*draft_salaries[p]
                tss[i] += dsal
                year_since_draft = i-(yr+1)
                x = draft_pick_movs[year_since_draft]
                pars[i].append(x[1]*np.exp(-x[0]*(p**x[2])))

    # add salaries and player values
    pred_movs = []
    for i in range(5):
        play = [p for p in pars[i] if p >= r_lvl]
        lp = len(play)
        if lp < 10:
            play= play + (10-lp)*[r_lvl]
        play = sorted(play,reverse=True)[:10]
        play_s = sum(play)
        salary_s = tss[i] + (10-lp)*minS
        cap_space = np.maximum(sCap-salary_s,0)/sCap
        ot = 1.0/3.0
        mov_from_cap = (cap_space//ot)*(sA) + (cap_space%ot)*sA

        x = player_vs_cap[i]
        p_mov = x[0] + x[1]*play_s + x[2]*mov_from_cap
        pred_movs.append(p_mov)


    # turn mov into win probs
    cA,cB = 0.46421272651304596, -5.12021285460421
    win_p = [1.0/(1+np.exp(-mov*cA -cB)) for mov in pred_movs]

    # discount factor for the future, more uncertainty, less sure reward
    value = [wp*(0.7*i) for i,wp in enumerate(win_p)] 
    return sum(value)
get_team_value(team,90000,750)

In [ ]:
data = json.load(open('real_2020.json','rt',encoding='utf-8-sig'))
gA = {_['key']:_['value'] for _ in data['gameAttributes']}

In [ ]:
team_players = defaultdict(list)
team_names = {}
season = gA['season']
sCap = gA['salaryCap']
minS = gA['minContract']

team_injuries = defaultdict(list)
for p in data['players']:
    if p['tid'] >= 0:
        ovr = p['ratings'][-1]['ovr']
        age = season - p['born']['year']
        salary = p['contract']['amount']
        years_left = p['contract']['exp']-season
        team_players[p['tid']].append((age,ovr,salary,years_left))
        team_injuries[p['tid']].append(p['injury']['gamesRemaining'])


In [ ]:
team_movs = defaultdict(float)

for t in data['teams']:
    tid = t['tid']
    team_names[t['tid']] = t['abbrev']
    for ts in t['stats']:
        current_mov = 0
        gp = ts['gp']
        gl = 82-gp
        if season != ts['season'] and not ts['playoffs']:
            mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
            current_mov = mov
        estimated_mov = sum(sorted([o2m[_[1]]*(max(gl-i,0)/gl) for _,i in zip(team_players[tid],team_injuries[tid])])[-10:])
        team_movs[tid] = (gp/82)*current_mov + (gl/82)*estimated_mov
    

In [ ]:
team_picks = defaultdict(list)
for d in data['draftPicks']:
    mov = team_movs[d['originalTid']]
    tid = d['tid']
    rnd = d['round']
    yl = d['season']-season
    team_picks[tid].append((rnd,yl,mov))

In [ ]:
teams_vals = []
for i in range(len(data['teams'])):
    val = get_team_value({'p':team_players[i],'d':team_picks[i]})
    teams_vals.append((val,team_names[i]))

In [ ]:
for v,t in sorted(teams_vals):
    print(v,t)